## Error Detection

### Importing

In [ ]:
import polars as pl

### Data File

In [ ]:
df = pl.read_csv(
    "pico_data.csv",
    schema_overrides={
        "trigger_data": pl.UInt32,
        "trigger_clk": pl.UInt64,
        "trigger_id": pl.UInt16,
        "veto_in": pl.Boolean,
        "internal_trigger": pl.Boolean,
    }
)
print(df)

### Time Conversion

In [ ]:
df = df.with_columns(
    (pl.col("trigger_clk") / 40000000).alias("trigger_clk")
)

#### Printing DataFrame

In [ ]:
print(df)

### Trigger Data Errors

In [ ]:
td_df = df.with_columns(
    pl.col("trigger_data").cast(pl.UInt32).map_elements(
        lambda x: format(x, '032b'),
        return_dtype=pl.String
    ).alias("trigger_data_binary")
)

for i in range(12):
    start = 8 + i * 2
    end = start + 2
    td_df = td_df.with_columns(
        td_df["trigger_data_binary"].str.slice(start, 2).map_elements(
            lambda x: int(x, 2),
            return_dtype=pl.UInt8
        ).alias(f"chunk_{i}")
    )

td_df = td_df.filter(
    pl.struct(td_df.columns).map_elements(lambda row: 0 not in row and 3 not in row)
)
    
td_df = td_df.drop("trigger_data_binary").drop("trigger_data")

msb_df = td_df.select(
    [f"chunk_{i}" for i in range(6)]
)

lsb_df = td_df.select(
    [f"chunk_{i}" for i in range(6, 12)]
)

#### Printing DataFrames

In [ ]:
print(msb_df)
print(lsb_df)

### Veto In Errors

In [ ]:
vi_df = df.filter(pl.col("veto_in") == True)

#### Printing DataFrames

In [ ]:
print(vi_df)

### Internal Trigger Errors

In [ ]:
it_df = df.filter(pl.col("internal_trigger") == True)

#### Printing DataFrames

In [ ]:
print(it_df)

## Plotting

In [ ]:
'''
-x = clk, y = faulty ids (tells us if there is an interval which issues happen)
-heat map (graph that changes color when get 0 or 2)
-3d gaussian distribution x = , y = , z = (tells us)
'''

### Faulty Rods at Intervals

### Heat Map

### Gaussian Distribution